In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
from sklearn.preprocessing import StandardScaler

In [3]:
x = np.load('/content/drive/MyDrive/data.T/X.npy')
y = np.load('/content/drive/MyDrive/data.T/Y.npy')

In [4]:
d = np.genfromtxt('/content/drive/MyDrive/data.T/Vert.csv', delimiter = ',')
d = d[1:]

In [6]:
dip_y = np.zeros((45006,3))
k=0
for i in range(15002):
  for j in range(3):
    dip_y[k] = d[y[i][j+1].nonzero()[0][0]]
    k+=1

In [7]:
pred_x1 = np.zeros((45006,309))

In [9]:
k=0
for i in range(15002):
  for j in range(3):
    pred_x1[k,:306] = x[i,j+1]
    pred_x1[k,306:309] = d[y[i][j].nonzero()[0][0]]
    k+=1

In [10]:
test_x = np.load('/content/drive/MyDrive/data.T/testX.npy')
test_y = np.load('/content/drive/MyDrive/data.T/testY.npy')

In [11]:
test_dip = np.zeros((test_y.shape[0]*3,3))
k=0
for i in range(test_y.shape[0]):
  for j in range(3):
    test_dip[k] = d[test_y[i][j+1].nonzero()[0][0]]
    k+=1

pred_test = np.zeros((test_x.shape[0]*3,309))
k=0
for i in range(test_x.shape[0]):
  for j in range(3):
    pred_test[k,:306] = test_x[i,j+1]
    pred_test[k,306:309] = d[test_y[i][j].nonzero()[0][0]]
    k+=1

In [14]:
scaler = StandardScaler().fit(pred_x1)
pred_x1 = scaler.transform(pred_x1)

In [16]:
scaler = StandardScaler().fit(pred_test)
pred_test = scaler.transform(pred_test)

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input
from keras.metrics import RootMeanSquaredError
from keras.layers import SimpleRNN
from keras import regularizers

In [79]:
model = Sequential()
model.add(Dense(150, kernel_regularizer=regularizers.L1L2(l1=0.01, l2=0.01),
                      bias_regularizer=regularizers.L2(0.01),
                      activity_regularizer=regularizers.L2(0.01)))
model.add(Dense(3))

In [80]:
model.compile(optimizer='SGD', loss='mean_absolute_error', metrics = RootMeanSquaredError())

In [81]:
model.fit(pred_x1, dip_y, epochs=10, batch_size=32)

Epoch 1/10
1407/1407 [==============================] - 4s 3ms/step - loss: 6.7286 - root_mean_squared_error: 0.2364
Epoch 2/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0896 - root_mean_squared_error: 0.0554
Epoch 3/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0748 - root_mean_squared_error: 0.0407
Epoch 4/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0632 - root_mean_squared_error: 0.0299
Epoch 5/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0543 - root_mean_squared_error: 0.0223
Epoch 6/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0476 - root_mean_squared_error: 0.0169
Epoch 7/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0431 - root_mean_squared_error: 0.0133
Epoch 8/10
1407/1407 [==============================] - 6s 4ms/step - loss: 0.0402 - root_mean_squared_error: 0.0111
Epoch 9/10
1407/1407 [==============================] - 4s 3ms/s

In [85]:
pred_test1 = model.predict(pred_test)

24/24 [==============================] - 0s 1ms/step


In [86]:
print(pred_test1[0])
print(test_dip[0])

[-0.04841579 -0.02272133  0.07340553]
[-0.046036 -0.032946  0.081814]


In [87]:
err = 0
for i in range(test_dip.shape[0]):
  err += (((test_dip[i][0]-pred_test1[i][0])*1000)**2+((test_dip[i][1]-pred_test1[i][1])*1000)**2+((test_dip[i][2]-pred_test1[i][2])*1000)**2)**(1/2)
print(err/750)

14.920412885701905


In [116]:
est_dip = np.zeros((test_dip.shape[0],3))

In [117]:
est_dip[0] = model.predict(pred_test[:1])

1/1 [==============================] - 0s 18ms/step


In [118]:
est_dip[0]

array([-0.04841579, -0.02272133,  0.07340553])

In [119]:
test_dip[0]

array([-0.046036, -0.032946,  0.081814])

In [120]:
buf = np.zeros((2,309))
i = 0
while i <= test_dip.shape[0]-1:
  est_dip[i] = model.predict(pred_test[i:i+1])
  i+=1
  for j in range(2):
    x = np.concatenate((pred_test[i][:306],est_dip[i-1]), axis = 0)
    buf[0] = x
    est_dip[i] = model.predict(buf[:1])
    i+=1 

1/1 [==============================] - 0s 13ms/step


In [113]:
i

750

In [121]:
err = 0
for i in range(est_dip.shape[0]):
  err += (((test_dip[i][0]-est_dip[i][0])*1000)**2+((test_dip[i][1]-est_dip[i][1])*1000)**2+((test_dip[i][2]-est_dip[i][2])*1000)**2)**(1/2)
print(err/750)

45.864957720797335
